## Setup for Inpainting Eraser Models
---
In this notebook, you are going to run a defined setup process for our inpainting eraser solution. Due to the size of the models, some of the step may take some time to complete. The entire notebook should finish within 1 hour. At the end of the notebook run, we should have a instended inference container in Elastic Container Registry (ECR) ready to host our models using SageMaker Endpoint (MME).

SageMaker MME is a service provided by Amazon SageMaker that allows multiple machine learning models to be hosted on a single endpoint. This means that multiple models can be deployed and managed together, making it easier to scale and maintain machine learning applications. With a multi-model endpoint, different models can be selected based on specific needs, allowing for more flexibility and efficiency. It also enables different types of models to be combined, such as computer vision and natural language processing models, to create more comprehensive applications.

Here is a high level breakdown of the setup steps:

1. Downloading pre-trained models
2. Package conda environment for additional model dependencies
3. Extend SageMaker managed Triton container with model checkpoints and conda packs pre-loaded
4. Push the container to AWS Elastic Container Registry (ECR)

---

This notebook will locally build a custom docker image. **We recommend to use pytorch kernel on SageMaker Notebook Instance using `ml.g4dn.xlarge`**

In [ ]:
!pip install -Uq sagemaker

### Setup

In [ ]:
import sagemaker
import boto3

import tarfile
import os

sagemaker_session = sagemaker.Session(boto_session=boto3.Session())
region = sagemaker_session.boto_region_name
account = sagemaker_session.account_id()

### Download Pre-trained Models

#### Download and Package SAM Checkpoint
**Apache-2.0 license**

In [ ]:
model_file_name = "sam_vit_h_4b8939.pth"
download_path = f"https://huggingface.co/spaces/abhishek/StableSAM/resolve/main/{model_file_name}"

!wget $download_path

In [ ]:
sd_tar = f"docker/{model_file_name}.tar.gz"

def make_tarfile(output_filename, source_dir):
    with tarfile.open(output_filename, "w:gz") as tar:
        tar.add(source_dir, arcname=os.path.basename(source_dir))

make_tarfile(sd_tar, model_file_name)

### Download and Package LaMa Checkpoint
**Apache-2.0 license**

In [ ]:
!wget https://huggingface.co/smartywu/big-lama/resolve/main/big-lama.zip
!unzip big-lama.zip

In [ ]:
lama_dir = "big-lama"
lama_tar = f"docker/{lama_dir}.tar.gz"

def make_tarfile(output_filename, source_dir):
    with tarfile.open(output_filename, "w:gz") as tar:
        tar.add(source_dir, arcname=os.path.basename(source_dir))

make_tarfile(lama_tar, lama_dir)

LaMa needs additional source code script. We will clone the repo into our `model_repo` folder

In [ ]:
!cd model_repo/lama/1 && git clone https://github.com/advimman/lama.git

#### Downloading Images and Modules from Inpaint Anything
**Apache-2.0 license**

In [ ]:
!cd statics && wget https://raw.githubusercontent.com/geekyutao/Inpaint-Anything/main/example/fill-anything/sample1.png
!cd statics && wget https://raw.githubusercontent.com/geekyutao/Inpaint-Anything/main/example/remove-anything/dog.jpg

### Package Conda Environment for each model

SageMaker NVIDIA Triton container images does not contain all the libraries two run our SAM and LaMa models. However, Triton allows you to bring additional dependencies using conda pack. Let's run the two cells below to create a `xxx_env.tar.gz` environment package for each model.

In [ ]:
!cd docker && bash sam_conda_dependencies.sh

In [ ]:
!cd docker && bash lama_conda_dependencies.sh

### Extend SageMaker Managed Triton Container

When we host these models on SageMaker MME. When invoked, model files will be loaded from S3 onto the instance. Due the large size of our models and model packages (SAM: 2.4GB + conda pack: 2.52 GB, LaMa: 0.38 GB + conda pack: 3.35GB), we are going to pre-load these files into the container. This will reduce model loading time and improve user experience during cold start.

In [ ]:
# account mapping for SageMaker Triton Image
account_id_map = {
    "us-east-1": "785573368785",
    "us-east-2": "007439368137",
    "us-west-1": "710691900526",
    "us-west-2": "301217895009",
    "eu-west-1": "802834080501",
    "eu-west-2": "205493899709",
    "eu-west-3": "254080097072",
    "eu-north-1": "601324751636",
    "eu-south-1": "966458181534",
    "eu-central-1": "746233611703",
    "ap-east-1": "110948597952",
    "ap-south-1": "763008648453",
    "ap-northeast-1": "941853720454",
    "ap-northeast-2": "151534178276",
    "ap-southeast-1": "324986816169",
    "ap-southeast-2": "355873309152",
    "cn-northwest-1": "474822919863",
    "cn-north-1": "472730292857",
    "sa-east-1": "756306329178",
    "ca-central-1": "464438896020",
    "me-south-1": "836785723513",
    "af-south-1": "774647643957",
}



region = boto3.Session().region_name
if region not in account_id_map.keys():
    raise ("UNSUPPORTED REGION")

base = "amazonaws.com.cn" if region.startswith("cn-") else "amazonaws.com"
mme_triton_image_uri = (
    "{account_id}.dkr.ecr.{region}.{base}/sagemaker-tritonserver:22.12-py3".format(
        account_id=account_id_map[region], region=region, base=base
    )
)
triton_account_id = account_id_map[region]
mme_triton_image_uri

Preview the docker file

In [ ]:
!cat docker/Dockerfile

### Build & push the new image to ECR

In [ ]:
# New container image name
new_image_name = 'sagemaker-tritonserver-sam-lama'

In [ ]:
%%capture build_output
!cd docker && bash build_and_push.sh "$new_image_name" "latest" "$mme_triton_image_uri" "$region" "$account" "$triton_account_id"

In [ ]:
if 'Error response from daemon' in str(build_output):
    print(build_output)
    raise SystemExit('\n\n!!There was an error with the container build!!')
else:
    extended_triton_image_uri = str(build_output).strip().split('\n')[-1]
    
print(f"New image URI from ECR: {extended_triton_image_uri}")

In [ ]:
%store extended_triton_image_uri